In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import tqdm

In [150]:
df = pd.read_csv("QUANTT_dataset.csv").iloc[:,1:]

In [151]:
df.head()

,Stock price,Strike price,T,Risk-Free Rate,Log Returns,Historical Volatility
0,9.9867,NaN,NaN,0.0300,-0.007245,0.306858
1,10.0000,NaN,NaN,0.0301,0.000578,0.306257
2,10.0000,NaN,NaN,0.0298,0.000000,0.305665
3,9.8413,NaN,NaN,0.0296,-0.006948,0.305211
4,9.9233,NaN,NaN,0.0301,0.003604,0.304616


In [152]:
df = pd.concat([df]*10).reset_index(drop=True)

In [153]:
# Generating random values for Strike price and T
np.random.seed(42)  # Setting a seed for reproducibility
df["Strike price"] = np.clip(np.random.normal(df["Stock price"], df["Stock price"]/4, len(df)), 0, 2 * df["Stock price"])
df["Implied Volatility"] = np.clip(np.random.gamma(2, 40, len(df)), 0.0001, None)
df["Implied Volatility"] = df["Implied Volatility"]/100
df["T"] = np.random.triangular(0, 0, 1, size=len(df))

In [154]:
def black_scholes(S, X, T, r, sigma):
    """
    S: Current stock price
    X: Strike price
    T: Time to maturity (in years)
    r: Risk-free interest rate (expressed as a decimal)
    sigma: Volatility of the stock's return
    """
    # Calculate d1 and d2
    d1 = (np.log(S / X) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
   
    # Calculate the call option price
    call_price = (S * norm.cdf(d1)) - (X * np.exp(-r * T) * norm.cdf(d2))
   
    # Calculate the put option price
    put_price = (X * np.exp(-r * T) * norm.cdf(-d2)) - (S * norm.cdf(-d1))
   
    return call_price

def monte_carlo(Stock_P, Strike_P, vol, r, T, N=10, M=1_000):
    # Constants. These apply to the formula -----> delta(x) = u*delta(t) + sigma*delta(z)
    dt = T/N                                #   timestep. I.E., if T is 5 years and N is 10, this will give each time step as 0.5 years
    u_dt = (r-0.5*vol**2)*dt                #   drift term (u)  
    vol_sqrt_dt = vol*np.sqrt(dt)           #   From equation on line 43, delta(z) becomes sqrt(delta(t)) when time step is applied
                                            #   -----> StockPriceNow = StockPricePrevious*e^(u*delta(t) + sigma*delta(z))
    # Standard error placeholders
    totalCT = 0
    totalCT2 = 0

    # MONTE CARLO METHOD
    for i in range(M):  #M is total simulations
        lnStock_P = np.log(Stock_P)
        for j in range(N):  #N is total timesteps
            lnStock_P = lnStock_P + u_dt + vol_sqrt_dt*np.random.normal()

        ST = np.exp(lnStock_P) # e^ln cancels out and leaves the stock price as ST
        CT = max(0, ST - Strike_P) 
        totalCT = totalCT + CT
        totalCT2 = totalCT2 +CT*CT

    # Find the call value and the SE
    C0 = np.exp(-r*T)*totalCT/M    # C0 is call value. Comes from formula ------> C0 = (1/M)*(SIGMA(  M(top of sigma)...i=1(bottom of sigma) C0 ))
    sigma = np.sqrt((totalCT2 - totalCT*totalCT/M)*np.exp(-2*r*T) / (M-1)) # for standard error. Has nothing to do with uppercase SIGMA from line 65
    SE = sigma/np.sqrt(M) #find the standard error. This is essentially based on number of sims and call value at each time point

    return C0

def binomial_tree(S0, K, T, r, sigma, N = 100):
    
    # Calculations
    dt = T / N  # length of each step
    u = np.exp(sigma * np.sqrt(dt))  # up-factor
    d = 1 / u  # down-factor
    p = (np.exp(r * dt) - d) / (u - d)  # risk-neutral probability

    # Initialize the Stock Price Tree
    stock_price_tree = np.zeros((N + 1, N + 1))
    stock_price_tree[0, 0] = S0
    for i in range(1, N + 1):
        stock_price_tree[i, 0] = stock_price_tree[i - 1, 0] * u
        for j in range(1, i + 1):
            stock_price_tree[i, j] = stock_price_tree[i - 1, j - 1] * d

    # Initialize the Option Value Tree for a Call Option
    option_value_tree = np.zeros((N + 1, N + 1))
    for j in range(N + 1):
        option_value_tree[N, j] = max(0, stock_price_tree[N, j] - K)

    # Perform Backward Induction
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            option_value_tree[i, j] = np.exp(-r * dt) * (p * option_value_tree[i + 1, j] + (1 - p) * option_value_tree[i + 1, j + 1])

    # Result
    option_price = option_value_tree[0, 0]

    return option_price

In [155]:
# Setting up a progress bar for the DataFrame's length
for i in tqdm.tqdm(df.index, desc='Calculating Values', position=0, leave=True):
    row = df.loc[i]
    df.at[i, 'BS Value'] = black_scholes(row['Stock price'], row['Strike price'], row['T'], row['Risk-Free Rate'], row['Implied Volatility'])
    df.at[i, 'MC Value'] = monte_carlo(row['Stock price'], row['Strike price'], row['Implied Volatility'], row['Risk-Free Rate'], row['T'])
    df.at[i, 'BT Value'] = binomial_tree(row['Stock price'], row['Strike price'], row['T'], row['Risk-Free Rate'], row['Implied Volatility'])

Calculating Values:   0%|          | 0/25770 [00:00<?, ?it/s]

Calculating Values: 100%|██████████| 25770/25770 [23:56<00:00, 17.94it/s] 


In [156]:
df['Average Value'] = df[['BT Value', 'BS Value', 'MC Value']].mean(axis=1)

In [157]:
df.head()

,Stock price,Strike price,T,Risk-Free Rate,Log Returns,Historical Volatility,Implied Volatility,BS Value,MC Value,BT Value,Average Value
0,9.9867,11.226834,0.337134,0.0300,-0.007245,0.306858,2.268051,4.618132,3.524582,4.627319,4.256678
1,10.0000,9.654339,0.247160,0.0301,0.000578,0.306257,0.787797,1.737644,1.879160,1.741240,1.786015
2,10.0000,11.619221,0.511068,0.0298,0.000000,0.305665,0.222180,0.186846,0.214373,0.187164,0.196128
3,9.8413,13.588448,0.091601,0.0296,-0.006948,0.305211,1.146393,0.382830,0.401781,0.384253,0.389622
4,9.9233,9.342406,0.135366,0.0301,0.003604,0.304616,2.919937,4.246359,4.267220,4.250949,4.254843


In [149]:
df.to_csv("modified_QUANTT_dataset.csv")